In [1]:
# Import Library
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import time
import pandas as pd

In [2]:
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome("chrome_driver\chromedriver.exe", options=options)

# 메인 홈페이지로 이동
driver.get("https://www.koreabaseball.com/")

# 기록 페이지로 이동
css_selector = "#lnb > li:nth-child(3)"
elem = driver.find_element(By.CSS_SELECTOR, css_selector)
elem.click()

# 기록실 페이지로 이동
css_selector = "#lnb02"
elem = driver.find_element(By.CSS_SELECTOR, css_selector)
elem.click()

# 기록실 옵션(타자, 투수, 수비, 주루)
options = ['hitter', 'pitcher', 'defense', 'runner']
record_tab_xpath = '//*[@id="contents"]/div[2]/div[2]/ul/*'

C:\Users\Luna\AppData\Local\Temp\ipykernel_11048\3091567411.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chrome_driver\chromedriver.exe", options=options)


In [5]:
record_tab = driver.find_elements(By.XPATH, record_tab_xpath)
record_tab[0].click()

time.sleep(3)

table_head = driver.find_element(By.CLASS_NAME, 'tData01').text.split('\n').pop(0).split(' ')

record_next = driver.find_element(By.CLASS_NAME, 'next')
record_next.click()

table_head2 = driver.find_element(By.CLASS_NAME, 'tData01').text.split('\n').pop(0).split(' ')
for i in range(0, 4):
    del table_head2[0]

table_head = table_head + table_head2

record_prev = driver.find_element(By.CLASS_NAME, 'prev')
record_prev.click()

table_value = []

print(table_head)

['순위', '선수명', '팀명', 'AVG', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'TB', 'RBI', 'SAC', 'SF', 'BB', 'IBB', 'HBP', 'SO', 'GDP', 'SLG', 'OBP', 'OPS', 'MH', 'RISP', 'PH-BA']


In [6]:
select = Select(driver.find_element(By.NAME, "ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlTeam$ddlTeam"))
select.select_by_value(value='OB')

time.sleep(3)

pages = driver.find_element(By.CLASS_NAME, 'paging').text.split(' ')

for page in range(1, len(pages)+1):
    id_selector = 'cphContents_cphContents_cphContents_ucPager_btnNo' + str(page)
    elem = driver.find_element(By.ID, id_selector)
    elem.click()

    time.sleep(3)

    temp = driver.find_element(By.CLASS_NAME, 'tData01').text.split('\n')
    del temp[0]
    table_value.extend(temp)

print(table_value)

['1 김태근 두산 0.500 2 6 4 0 2 1 0 0 3 2 0 0', '2 강현구 두산 0.333 3 5 3 0 1 0 0 0 1 0 0 0', '2 안승한 두산 0.333 30 39 36 5 12 3 0 0 15 8 1 0', '4 페르난데스 두산 0.309 139 550 508 52 157 28 0 6 203 77 0 5', '5 안권수 두산 0.297 76 267 239 43 71 7 2 0 82 20 1 0', '6 허경민 두산 0.289 121 493 432 59 125 23 0 8 172 60 2 7', '7 전민재 두산 0.289 35 46 45 11 13 1 0 0 14 0 1 0', '8 김민혁 두산 0.284 38 97 88 11 25 2 0 5 42 16 0 0', '9 강승호 두산 0.264 134 487 444 54 117 28 1 10 177 62 6 5', '10 정수빈 두산 0.259 127 455 405 58 105 12 4 3 134 41 6 4', '11 송승환 두산 0.250 11 28 28 3 7 1 0 1 11 4 0 0', '12 박세혁 두산 0.248 128 402 351 33 87 13 1 3 111 41 8 4', '13 김재환 두산 0.248 128 517 448 64 111 24 1 23 206 72 0 4', '14 김인태 두산 0.247 83 279 235 25 58 6 0 5 79 25 0 0', '15 양석환 두산 0.244 107 446 405 58 99 14 1 20 175 51 0 2', '16 양찬열 두산 0.244 41 94 82 15 20 4 0 3 33 12 1 0', '17 이유찬 두산 0.241 13 31 29 3 7 0 0 1 10 2 1 1', '18 김대한 두산 0.240 51 110 96 13 23 6 1 4 43 11 2 1', '19 박유연 두산 0.235 13 19 17 0 4 0 0 0 4 2 0 0', '19 조수행 두산 0.235 117 134 119 36 28 

In [7]:
record_next = driver.find_element(By.CLASS_NAME, 'next')
record_next.click()

time.sleep(3)

table_value2 = []

pages = driver.find_element(By.CLASS_NAME, 'paging').text.split(' ')

for page in range(1, len(pages)+1):
    id_selector = 'cphContents_cphContents_cphContents_ucPager_btnNo' + str(page)
    elem = driver.find_element(By.ID, id_selector)
    elem.click()

    time.sleep(3)

    temp = driver.find_element(By.CLASS_NAME, 'tData01').text.split('\n')
    del temp[0]
    table_value2.extend(temp)

print(table_value2)

['1 김태근 두산 0.500 1 0 1 1 0 0.750 0.667 1.417 0 1.000 0.000', '2 강현구 두산 0.333 1 0 1 0 0 0.333 0.600 0.933 0 0.000 0.000', '2 안승한 두산 0.333 2 0 0 12 0 0.417 0.368 0.785 2 0.385 1.000', '4 페르난데스 두산 0.309 35 0 2 45 34 0.400 0.353 0.753 43 0.343 0.500', '5 안권수 두산 0.297 26 0 1 40 6 0.343 0.368 0.711 19 0.320 0.000', '6 허경민 두산 0.289 36 3 16 40 12 0.398 0.360 0.758 33 0.317 0.000', '7 전민재 두산 0.289 0 0 0 12 3 0.311 0.289 0.600 1 0.000 0.000', '8 김민혁 두산 0.284 7 0 2 25 6 0.477 0.351 0.828 5 0.320 0.294', '9 강승호 두산 0.264 29 0 3 100 8 0.399 0.310 0.709 34 0.299 0.250', '10 정수빈 두산 0.259 39 1 1 56 2 0.331 0.323 0.654 29 0.237 0.000', '11 송승환 두산 0.250 0 0 0 7 0 0.393 0.250 0.643 1 0.182 0.667', '12 박세혁 두산 0.248 33 1 6 71 4 0.316 0.320 0.636 18 0.354 0.200', '13 김재환 두산 0.248 61 4 4 133 5 0.460 0.340 0.800 22 0.271 0.333', '14 김인태 두산 0.247 41 1 3 47 6 0.336 0.366 0.702 11 0.246 0.250', '15 양석환 두산 0.244 32 3 7 101 3 0.432 0.309 0.741 23 0.245 0.000', '16 양찬열 두산 0.244 11 0 0 23 2 0.402 0.333 0.735 3 0.238 

In [8]:
temp_values = []
temp_values2 = []

for item, item2 in zip(table_value, table_value2):
    temp_values.append(item.split(' '))
    temp_values2.append(item2.split(' '))

for i in range(0, len(temp_values2)):
    for j in range(0, 4):
        del temp_values2[i][0]

record_values = []

for item, item2 in zip(temp_values, temp_values2):
    record_values.append(item + item2)

print(record_values)


[['1', '김태근', '두산', '0.500', '2', '6', '4', '0', '2', '1', '0', '0', '3', '2', '0', '0', '1', '0', '1', '1', '0', '0.750', '0.667', '1.417', '0', '1.000', '0.000'], ['2', '강현구', '두산', '0.333', '3', '5', '3', '0', '1', '0', '0', '0', '1', '0', '0', '0', '1', '0', '1', '0', '0', '0.333', '0.600', '0.933', '0', '0.000', '0.000'], ['2', '안승한', '두산', '0.333', '30', '39', '36', '5', '12', '3', '0', '0', '15', '8', '1', '0', '2', '0', '0', '12', '0', '0.417', '0.368', '0.785', '2', '0.385', '1.000'], ['4', '페르난데스', '두산', '0.309', '139', '550', '508', '52', '157', '28', '0', '6', '203', '77', '0', '5', '35', '0', '2', '45', '34', '0.400', '0.353', '0.753', '43', '0.343', '0.500'], ['5', '안권수', '두산', '0.297', '76', '267', '239', '43', '71', '7', '2', '0', '82', '20', '1', '0', '26', '0', '1', '40', '6', '0.343', '0.368', '0.711', '19', '0.320', '0.000'], ['6', '허경민', '두산', '0.289', '121', '493', '432', '59', '125', '23', '0', '8', '172', '60', '2', '7', '36', '3', '16', '40', '12', '0.398', '0.